> 原文地址 [www.cnblogs.com](https://www.cnblogs.com/xleng/p/11791944.html)

# 摘要


　　GStreamer 提供了不同的命令行工具用于快速的查看信息以及验证 Pipeline 的是否能够正确运行，在平时的开发过程中，我们也优先使用 GStreamer 的命令行工具验证，再将 Pipeline 集成到应用中。本文将介绍 gst-inspect-1.0，gst-discoverer-1.0，gst-launch-1.0 命令行工具的使用。

# gst-inspect-1.0

此命令有 3 种工作模式，实际中我们常用第一种和第三种方式：

## 1. 不带任何参数

这样会列出当前系统中支持的所有 Element，这些 Element 可用于构造 Pipeline.

## 2. 跟文件名

这样会将指定文件作为一个 GStreamer 插件，尝试列出其中所包含的 Element。例如下面的命令列出了 libgstjpeg.so 所包含的 2 个 Elements：

```sh
$ gst-inspect-1.0 /usr/lib/x86\_64-linux-gnu/gstreamer-1.0/libgstjpeg.so
Plugin Details:
  Name                     jpeg
  Description              JPeg plugin library
  Filename                 /usr/lib/x86\_64-linux-gnu/gstreamer-1.0/libgstjpeg.so
  Version                  1.8.3
  License                  LGPL
  Source module            gst-plugins-good
  Source release date      2016-08-19
  Binary package           GStreamer Good Plugins (Ubuntu)
  Origin URL               https://launchpad.net/distros/ubuntu/+source/gst-plugins-good1.0

  jpegenc: JPEG image encoder
  jpegdec: JPEG image decoder

  2 features:
  +-- 2 elements
```

## 3. 跟 Element 名

会列出 Element 的详细信息。例如，下面的命令会列出 jpeg 解码器所支持的输入数据类型，Pad 信息，支持的属性及值。我们主要关注 Pad Template，以及 Element 的属性信息。

```sh
$ gst-inspect-1.0 jpegdec
Factory Details:
  Rank                     primary (256)
  Long-name                JPEG image decoder
  Klass                    Codec/Decoder/Image
  Description              Decode images from JPEG format
  Author                   Wim Taymans <wim@fluendo.com>

Plugin Details:
  Name                     jpeg
  Description              JPeg plugin library
  Filename                 /usr/lib/x86\_64-linux-gnu/gstreamer-1.0/libgstjpeg.so
  Version                  1.8.3
  License                  LGPL
  Source module            gst-plugins-good
  Source release date      2016-08-19
  Binary package           GStreamer Good Plugins (Ubuntu)
  Origin URL               https://launchpad.net/distros/ubuntu/+source/gst-plugins-good1.0

GObject
 +----GInitiallyUnowned
       +----GstObject
             +----GstElement
                   +----GstVideoDecoder
                         +----GstJpegDec

Pad Templates:
  SINK template: 'sink'
    Availability: Always
    Capabilities:
      image/jpeg

  SRC template: 'src'
    Availability: Always
    Capabilities:
      video/x-raw
                 format: { I420, RGB, BGR, RGBx, xRGB, BGRx, xBGR, GRAY8 }
                  width: \[ 1, 2147483647 \]
                 height: \[ 1, 2147483647 \]
              framerate: \[ 0/1, 2147483647/1 \]


Element Flags:
  no flags set

Element Implementation:
  Has change\_state() function: gst\_video\_decoder\_change\_state

Element has no clocking capabilities.
Element has no indexing capabilities.
Element has no URI handling capabilities.

Pads:
  SINK: 'sink'
    Implementation:
      Has chainfunc(): gst\_video\_decoder\_chain
      Has custom eventfunc(): gst\_video\_decoder\_sink\_event
      Has custom queryfunc(): gst\_video\_decoder\_sink\_query
      Has custom iterintlinkfunc(): gst\_pad\_iterate\_internal\_links\_default
    Pad Template: 'sink'
  SRC: 'src'
    Implementation:
      Has custom eventfunc(): gst\_video\_decoder\_src\_event
      Has custom queryfunc(): gst\_video\_decoder\_src\_query
      Has custom iterintlinkfunc(): gst\_pad\_iterate\_internal\_links\_default
    Pad Template: 'src'

Element Properties:
  name                : The name of the object
                        flags: readable, writable
                        String. Default: "jpegdec0"
  parent              : The parent of the object
                        flags: readable, writable
                        Object of type "GstObject"
  idct-method         : The IDCT algorithm to use
                        flags: readable, writable
                        Enum "GstIDCTMethod" Default: 1, "ifast"
                           (0): islow            - Slow but accurate integer algorithm
                           (1): ifast            - Faster, less accurate integer method
                           (2): float            - Floating-point: accurate, fast on fast HW
  max-errors          : (Deprecated) Error out after receiving N consecutive decoding errors (-1 = never fail, 0 = automatic, 1 = fail on first error)
                        flags: readable, writable, 0x80000000
                        Integer. Range: -1 - 2147483647 Default: 0
```

# gst-discoverer-1.0

　　此命令在 [GStreamer 基础教程 06 - 获取媒体信息](https://www.cnblogs.com/xleng/p/11277397.html)中使用过，是对 [GstDiscoverer](https://gstreamer.freedesktop.org/documentation/pbutils/gstdiscoverer.html?gi-language=c) 接口的封装，可以方便的查看媒体文件的编码，帧率等信息。


```sh
$ gst-discoverer-1.0 sintel\_trailer-480p.mp4
Analyzing file:///home/xleng/video/sintel\_trailer-480p.mp4
Done discovering file:///home/xleng/video/sintel\_trailer-480p.mp4

Topology:
  container: Quicktime
    audio: MPEG-4 AAC
    video: H.264 (High Profile)

Properties:
  Duration: 0:00:52.209000000
  Seekable: yes
  Live: no
  Tags:
      audio codec: MPEG-4 AAC audio
      maximum bitrate: 128000
      datetime: 1970-01-01T00:00:00Z
      title: Sintel Trailer
      artist: Durian Open Movie Team
      copyright: (c) copyright Blender Foundation | durian.blender.org
      description: Trailer for the Sintel open movie project
      encoder: Lavf52.62.0
      container format: ISO MP4/M4A
      video codec: H.264 / AVC
      bitrate: 535929

```

# gst-launch-1.0

　　gst-launch 是我们平时使用最多的一个命令，它接收一个用字符串方式描述的 Pipline，将其实例化并运行。我们可以用此命令快速的检查 Pipeline 中各个元素是否能够正确的连接起来。当我们需要构建的 Pipeline 很复杂时，我们也可以将 Pipeline 进行拆分，逐步通过 gst-launch 验证 Pipeline 的合法性。  
　　通过 gst-launch 验证的字符串 Pipeline 可以直接使用 gst\_parse\_launch() 接口将其转化为 GstPipeline 对象，节省了我们单独调用 API 去创建 Element 的时间。  
　　当我们用字符串描述 Pipeline 时，每个 Element 之间需要通过叹号 “!" 分隔 Element，这样 gst-launch 才能正确识别。  
　　在使用 gst-launch 时，根据不同的应用场景，我们可以分为以下的类型。

采用默认的参数创建 Pipeline
------------------

　　这种方式我们只需将所使用的 Element 使用叹号分隔即可，例如：

```sh
$ gst-launch-1.0 videotestsrc ! videoconvert ! autovideosink
```

# 设置 Element 的属性

　　在某些情况下，我们需要修改 Element 的属性, 或指定所需参数（例如 playbin 的 uri 参数），Element 的属性直接跟在 Element 之后。下面的命令会设置 videotestsrc 的视频模式，输出图像为环形。属性支持的值可以通过 gst-inspect 命令查看。

```sh
$ gst-launch-1.0 videotestsrc pattern=11 ! videoconvert ! autovideosink
```

　　如果属性值中包含空格，我们可以将其置于单引号或双引号中。

```sh
$ gst-launch-1.0 -v videotestsrc ! clockoverlay halignment=right valignment=bottom text="Edge City" shaded-background=true font-desc="Sans, 36" ! videoconvert ! autovideosink

```

# 包含分支的 Pipeline

　　每个 Element 都有 name 的属性，我们可以利用 name 来实现包含多个分支的复杂的 Pipeline，这常见于有多个输出 / 输入的 Element（mux，demux，tee 等）。  
　　下面的命令通过 tee 创建了 2 个分支，分别用于不同的 sink，在一个分支是 Pipeline 完成后（到达 sink），我们可以使用 “name 加一个点号” 重新创建一个分支。

```sh
$ gst-launch-1.0 videotestsrc ! videoconvert ! tee name=t ! queue ! autovideosink t. ! queue ! autovideosink
```

　　使用同样的方式，我们也可以将多个分支合并为一个。下面的命令首先解码文件，将视频编码为 H.264，音频编码为 MP3，最终再合并生成 TS 文件。注意 dmux 和 mux Element 的使用。

```sh
$ gst-launch-1.0 filesrc location=surround.mp4 ! decodebin name=dmux ! queue ! audioconvert ! lamemp3enc ! mux. \\
```

```sh
dmux. ! queue ! x264enc ! mpegtsmux name=mux ! queue ! filesink location=out.ts
```

# 指定 Element 的 Pad

　　某些情况下，我们希望自己指定某个 Pad 用于连接，我们可以指定已命名 Element 的 Pad 来实现。

```sh
$ gst-launch-1.0 souphttpsrc location=https://www.freedesktop.org/software/gstreamer-sdk/data/media/sintel\_trailer-480p.webm ! \\
matroskademux name=d d.video\_00 ! matroskamux ! filesink location=sintel\_video.mkv
```

　　上面的命令会将 sintel\_trailer-480p.webm 文件进行 Demux，只选择其中的视频 (video\_00)，再将其保留为 mk 文件，只保留了视频部分。如果想只保留音频部分，可以使用如下命令：

```sh
$ gst-launch-1.0 souphttpsrc location=https://www.freedesktop.org/software/gstreamer-sdk/data/media/sintel\_trailer-480p.webm ! \\
matroskademux name=d d.audio\_00 ! vorbisparse ! matroskamux ! filesink location=sintel\_audio.mka
```

　　上面两条命令均未对视频进行解码，只是将其从一个容器中拷入到另一个容器。

# 利用 Caps Filter 过滤码流

　　当 Element 包含多个输出 Pad 时，可能导致连接到下一个 Element 的 Pad 具有不确定性。在下一个 Element 支持当前 Element 所有的输出类型，这时 GStreamer 会随机选择一个 Pad 用于连接。  
　　例如：我们无法确定下面的 Pipeline 会使用 video\_00 还是 audio\_00 连接到 filesink，因为 filesink 同时支持 audio 及 video 输入。

```sh
$ gst-launch-1.0 souphttpsrc location=https://www.freedesktop.org/software/gstreamer-sdk/data/media/sintel\_trailer-480p.webm ! matroskademux ! filesink location=test

```

　　我们可以如上一节所说，显式指定连接所用的 Pad，或者使用 Caps Filter：

```sh
$ gst-launch-1.0 souphttpsrc location=https://www.freedesktop.org/software/gstreamer-sdk/data/media/sintel\_trailer-480p.webm ! \\
matroskademux ! video/x-vp8 ! matroskamux ! filesink location=sintel\_video.mkv
```

　　Caps Filter 表现为一个只接收指定数据类型的 Element，并将数据传递到下一个 Element，可以高效的解决二义性的问题。  
　　我们可以使用 gst-inspect 查看 Element 的输出 Pad，以决定我们的 Caps Filter 需要添加何种参数。在 gst-launch 后加 -v 参数可以输出 Pipeline 连接时所使用的 Pad 信息。  
　　对于多个过滤条件，我们需要通过逗号隔开。

　　对于某些特殊的类型参数，GStreamer 提供了相应的关键字来进行转换：

*   i 或 int 表示整型或范围；
*   f 或 float 表示浮点数及范围；
*   4 或 fourcc 表示 4 个字符的 FOURCC 代码；
*   b, bool, 或 boolean 表示布尔型数据；
*   s, str, 或 string 表示字符串；
*   fraction 表示分数（例如帧率， 像素宽高比）；
*   l 或 list 表示列表。

用于发送和接收 UDP RTP 数据：

```sh
$ gst-launch-1.0 v4l2src ! \\
video/x-raw-yuv,width=128,height=96,format='(fourcc)'UYVY ! \\
videoconvert ! ffenc\_h263 ! video/x-h263 ! rtph263ppay pt=96 ! \\
udpsink host=192.168.1.1 port=5000 sync=false

$gst-launch-1.0 udpsrc port=5000 ! application/x-rtp, \\
clock-rate=90000,payload=96 ! rtph263pdepay queue-delay=0 ! ffdec\_h263 \\
! xvimagesink
```

　　使用 YUY2 或 YV12 作为测试视频格式，帧率为 30 帧 / 秒：

```sh
$ gst-launch-1.0 videotestsrc ! \\
'video/x-raw-yuv,format=(fourcc)YUY2,framerate=30/1;video/x-raw-yuv,format=(fourcc)YV12,framerate=30/1' \\
! xvimagesink
```

　　通过 alsasrc 录制文件，限定采样率及位宽：

```sh
$ gst-launch-1.0 alsasrc! \\
'audio/x-raw-int,rate=\[32000,64000\],width=\[16,32\],depth={16,24,32},signed=(boolean)true' \\
! wavenc ! filesink location=recording.wav
```

# 总结

　　通过本文，我们掌握了：

*   如何通过 gst-inspect-1.0 查看当前系统所拥有的 GStreamer 插件以及每个插件的详细信息。
*   如何通过 gst-discoverer-1.0 查看媒体文件的 Metadata。
*   如何通过 gst-launch-1.0 运行及测试 Pipeline。
*   如果通过 Caps Filter 过滤指定数据。

# 引用

https://gstreamer.freedesktop.org/documentation/tools/gst-launch.html?gi-language=c  
https://gstreamer.freedesktop.org/documentation/tutorials/basic/gstreamer-tools.html?gi-language=c

作者：[John.Leng](http://www.cnblogs.com/xleng/) 出处：[http://www.cnblogs.com/xleng/](http://www.cnblogs.com/xleng/) 本文版权归作者所有，欢迎转载。商业转载请联系作者获得授权，非商业转载请在文章页面明显位置给出原文连接.